# Development of Tank Blending System Models

In [ ]:
import casadi as cas
import numpy as np
import matplotlib.pyplot as plt

from cas_models.continuous_time.models import (
    StateSpaceModelCT, 
    make_sim_step_function_RK4,
    make_n_step_simulation_function
)
from cas_models.discrete_time.models import StateSpaceModelDT

## Surge Tank with Mass Concentration

In [ ]:
# Physical constants
D = 5  # tank diameter [m]
A = np.pi * D  # tank cross-sectional area [m^2]

# System states: 
#  x[0] : Tank level, L [m]
#  x[1] : Total mass of suspended mineral in tank, m [tons]
n = 2

# Inputs
#  u[0] : volumetric flowrate into tank, v_dot_in [m^3/hr]
#  u[1] : density of fluid entering tank, rho_in [tons/m^3]
#  u[2] : volumetric flowrate out of tank, v_dot_out [m^3/hr]
nu = 3

# Outputs
#  y[0] : Tank level, L [m]
#  y[1] : Total mass of suspended mineral in tank, m [tons]
ny = 2

# Define the ODE right-hand side
x = cas.MX.sym('x', n)  
u = cas.MX.sym('u', nu)

dL_dt = (u[0] - u[2]) / A
dm_dt = u[0] * u[1] - u[2] * x[1] / (x[0] * A)

rhs = cas.vertcat(dL_dt, dm_dt)
assert rhs.shape == (n, 1)

t = cas.MX.sym('t')
f = cas.Function(
    "f",
    [t, x, u],
    [rhs],
    ["t", "x", "u"],
    ["rhs"]
)

# Define output function
y = x

h = cas.Function(
    "h",
    [t, x, u],
    [y],
    ["t", "x", "u"],
    ["y"]
)

print(f)
print(h)

In [ ]:
tank_model_ct = StateSpaceModelCT(f, h, n, nu, ny)
tank_model_ct

In [ ]:
F = make_sim_step_function_RK4(
    tank_model_ct.f, 
    tank_model_ct.h, 
    tank_model_ct.n, 
    tank_model_ct.nu, 
    params=tank_model_ct.params
)
F

In [ ]:
# Output function in discrete time is the same function
H = h

In [ ]:
# Test 1 - no change in volume or concentration
L = 5
conc = 0.2
m = A * L * conc
v_dot_in = 1
v_dot_out = 1

# Function arguments
t = 0.0
dt = 0.25
xk = cas.vertcat(L, m)
uk = cas.vertcat(v_dot_in, conc, v_dot_out)
xkp1 = F(t, xk, uk, dt)
print(xkp1)
assert np.allclose(xkp1, [[L], [m]])

# Check output function
yk = H(t, xk, uk)
print(yk)
assert np.allclose(yk, xk)

In [ ]:
# Test 2 - increasing volume
L = 5
conc = 0.2
m = A * L * conc
v_dot_in = 2
v_dot_out = 1

# Function arguments
t = 0.0
dt = 0.25
xk = cas.vertcat(L, m)
uk = cas.vertcat(v_dot_in, conc, v_dot_out)
xkp1 = F(t, xk, uk, dt)
L2 = L + (v_dot_in - v_dot_out) * dt / A
m2 = (A * L + (v_dot_in - v_dot_out) * dt) * conc
print(xkp1)
assert np.allclose(xkp1, [[L2], [m2]])

In [ ]:
# Test 3 - increasing concentration with no out flow
L = 5
conc = 0.2
m = A * L * conc
v_dot_in = 1
conc_in = 2 * conc
v_dot_out = 0

# Function arguments
t = 0.0
dt = 0.25
xk = cas.vertcat(L, m)
uk = cas.vertcat(v_dot_in, conc, v_dot_out)
xkp1 = F(t, xk, uk, dt)
L2 = L + (v_dot_in - v_dot_out) * dt / A
m2 = (A * L * conc + (v_dot_in * dt) * conc_in)
print(xkp1, [L2, m2])
assert np.allclose(xkp1, [[L2], [m2]], atol=0.1)  # TODO: figure out why this is not close

In [ ]:
# Test 4 - response with zero initial concentration in tank
L = 5
m = 0
v_dot_in = 1
conc_in = 0.5
v_dot_out = 1

# Function arguments
t = 0.0
dt = 0.25
xk = cas.vertcat(L, m)
uk = cas.vertcat(v_dot_in, conc_in, v_dot_out)
nT = 1000
X = np.full((nT+1, 2), np.nan)
for k in range(nT+1):
    xk = F(t, xk, uk, dt)
    X[k, :] = xk.T
    t += dt
m_final = A * L * conc_in
print(xk)
assert np.allclose(xk, [[L], [37.64707001]])

In [ ]:
t = dt * np.arange(0, nT+1)
plt.plot(t, X)
plt.hlines(m_final, t[0], t[-1], color='r', linestyle='--')
plt.grid()
plt.xlabel('Time $t$')
plt.ylabel(r'Output Variables')
plt.legend(labels=['Tank level', 'Total mass', 'Mass in steady-state'], loc='best')
plt.show()

In [ ]:
nT = 1000
simulate = make_n_step_simulation_function(F, H, n, nu, ny, nT, params=tank_model_ct.params)
simulate

In [ ]:
Ts = 0.25
t_eval = Ts * np.arange(nT+1)
U = np.full((nT, nu), np.nan)

v_dot_in = 1
conc_in = 0.5
v_dot_out = 1
U[:, 0] = v_dot_in
U[:, 1] = conc_in
U[:, 2] = v_dot_out
assert U.shape == (nT, nu)

L = 5
m = 0
x0 = [L, m]

assert t_eval.shape == (nT + 1, )
X, Y = simulate(t_eval, U, x0)

In [ ]:
t = dt * np.arange(0, nT+1)
plt.plot(t, X)
plt.hlines(m_final, t[0], t[-1], color='r', linestyle='--')
plt.grid()
plt.xlabel('Time $t$')
plt.ylabel(r'Output Variables')
plt.legend(labels=['Tank level', 'Total mass', 'Mass in steady-state'], loc='best')
plt.show()